In [1]:
import pandas as pd
import sys
from top2vec import Top2Vec
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import punkt
from nltk.stem import WordNetLemmatizer
from collections import Counter
# import umap.umap_ as umap
# import umap.plot
nltk.download("stopwords")
import numpy as np
import json
import glob
import gensim
import gensim.corpora as corpora 
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import TfidfModel
import spacy
from nltk.corpus import stopwords
import pyLDAvis
import pyLDAvis.gensim_models
from bertopic import BERTopic

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\plotly\graph_objs\__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\plotly\graph_objs\__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages

In [2]:

# def explore_common_words(df):
#     stop = stopwords.words("english")
#     df = df.astype(str)
#     df["allegation_desc"] = df["allegation_desc"].apply(
#         lambda x: " ".join([word for word in x.split() if word not in (stop)])
#     )

#     df.loc[:, "topics"] = " ".join(x for x in df["allegation_desc"].astype(str))
#     df = df["topics"][0]
#     df = df.split()

#     counts = Counter(df)
#     most_occur = counts.most_common(50)
#     print(most_occur)
#     return df

In [3]:
# def preprocess_nopd(df):
#     df.loc[:, "allegation_desc"] = df.allegation_desc.astype(str).fillna("")\
#         .str.replace(r"( ?officers? | ?complain?a?n?ts?\.? ?| ?stated ?| ?accused ?| ?nan ?| ?police ?)", "", regex=True)
#     return df

In [4]:
def split_data():
    df = pd.read_csv("../data/training_data/complaint_classification_labeled_data_1_31_2023.csv")

    df.loc[:, "label"] = df.label.str.lower().str.strip().str.replace(r"#", "/", regex=False)
    
    dfa = df
    dfa.loc[:, "target"] = df.label.str.lower().str.strip().str.replace(r"internal misconduct\/administrative infractions|harm to the general public safety", "", regex=True)
    dfa = dfa[~((dfa.target.fillna("") == ""))]
    dfa.loc[:, "target"] = dfa.target.str.replace(r"(.+)", "0", regex=True)

    targets = df.label.str.lower().str.strip().str.extract(r"(internal misconduct\/administrative infractions|harm to the general public safety)")

    df.loc[:, "target"] = targets[0].str.replace(r"(.+)", "1", regex=True)
    df = df[~((df.target.fillna("") == ""))]

    df = pd.concat([df, dfa], axis=0)

    df = df.rename(columns={"text": "allegation_desc"})

    df = df.astype(str)
    df = df.sample(frac=1)
    
    training_data = df[:700]
    test_data = df[700:]
    return training_data, test_data, df

In [5]:
training_data, test_data, df = split_data()
training_data.to_csv("training_data/training_data.csv")
test_data.to_csv("test_data/test_data.csv")

In [6]:
################################################### top2vec ##################################################################################

In [7]:
def create_model(df):
    # unique = [x for x in df["allegation_topic_uid"]]
    df = [x for x in df["allegation_desc"]]

    model = Top2Vec(
        df,
        ngram_vocab=True,
        speed="deep-learn",
        use_embedding_model_tokenizer=True,
        min_count=5,
        # document_ids=unique
    )
    return model

In [8]:
def model():
    top2vec = create_model(df)
    return top2vec

In [9]:
model = model()

2023-01-31 15:27:02,272 - top2vec - INFO - Pre-processing documents for training
2023-01-31 15:27:02,354 - top2vec - INFO - Creating joint document/word embedding
2023-01-31 15:27:27,615 - top2vec - INFO - Creating lower dimension embedding of documents
2023-01-31 15:27:46,935 - top2vec - INFO - Finding dense areas of documents
2023-01-31 15:27:46,995 - top2vec - INFO - Finding topics


In [10]:
if len(model.get_topic_sizes()) > 1:
    topic_words, word_scores, topic_nums = model.get_topics()
    for words, scores, num in zip(topic_words, word_scores, topic_nums):
        print(num)
        print(f"Words: {words}")

0
Words: ['an' 'complainant' 'officer' 'not' 'with' 'accused' 'using' 'in' 'of'
 'while' 'documented' 'phone' 'complaint' 'who' 'action' 'to'
 'altercation' 'for' 'off' 'no' 'did' 'on' 'criminal' 'found' 'during'
 'the' 'out' 'violation' 'her' 'proper' 'incident' 'taken' 'supervisor'
 'parish' 'deputy' 'which' 'failed' 'was' 'assigned' 'has' 'arrested'
 'that' 'sustained' 'about' 'it' 'his' 'contact' 'against' 'rude' 'by']
1
Words: ['body worn' 'activate' 'worn' 'uof' 'camera' 'end' 'body' 'uof incident'
 'an inmate' 'inmates' 'battery' 'medical' 'shift' 'completing' 'during'
 'spray' 'firearm' 'oc' 'tour' 'inmate' 'instructed' 'phone' 'assignment'
 'revealed' 'tone' 'failed' 'language' 'opso' 'speaking' 'all' 'through'
 'assigned' 'record' 'pursuit' 'appear' 'bwc' 'son' 'pod' 'handling'
 'cell' 'using' 'girlfriend' 'zone' 'calls' 'before' 'observed' 'log'
 'investigation' 'turned' 'due']
2
Words: ['failed to' 'complainant stated' 'rude and' 'stated that' 'late for'
 'failing to' 'for 

In [11]:
topic_sizes, top_nums = model.get_topic_sizes()
print(topic_sizes)
print(top_nums)

[434 163 103]
[0 1 2]


In [12]:
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=1, num_docs=10)

for doc, score, doc_id in list(zip(documents, document_scores, document_ids)):
    print(f"Document: {doc_id}, Score: {score}")
    print("--------------------")
    print(doc)
    print("--------------------")

Document: 580, Score: 0.9481993913650513
--------------------
complainant alleged that officer failed to activate body worn camera.
--------------------
Document: 64, Score: 0.9368914365768433
--------------------
failed to activate her body worn camera during a uof incident # o000011821.
--------------------
Document: 619, Score: 0.9272060990333557
--------------------
failed to activate his body worn camera during a uof incident #o000011821.
--------------------
Document: 696, Score: 0.9164046049118042
--------------------
failed to activate body worn camera for uof incident failed to activate.
--------------------
Document: 248, Score: 0.9050106406211853
--------------------
accused failed to activate in-car camera.
--------------------
Document: 507, Score: 0.8490033149719238
--------------------
officer did not use a body worn camera.
--------------------
Document: 356, Score: 0.8458248376846313
--------------------
officer failed to activate his body-worn camera during a call for

In [13]:
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=0, num_docs=20)

ents = {(doc, score) for doc, score in list(zip(documents, document_scores))}
df = pd.DataFrame(ents, columns=["allegation_desc", "score"])
print(df)
# df.loc[:, "topic"] = "9"

                                      allegation_desc     score
0   accused of not completing her two cell inspect...  0.431933
1   accused of not reporting to duty on sept 3, 20...  0.427347
2   observed via camera a movie on watching a lapt...  0.462734
3   complainant stated the officer should have bee...  0.447992
4           handled by chief, no formal investigation  0.488557
5    officers are accused of using unauthorized force  0.405262
6   complainant alleged officers would not write a...  0.404368
7          left his assigned area without permission.  0.540990
8          left his assigned area without permission.  0.507012
9   deputy scott did not active his bwc during his...  0.444346
10  officer under his supervision that the officer...  0.427159
11  sergeant johnson approved two sercurity patrol...  0.531714
12  engaged in a physical altercation inside a nig...  0.412561
13  officer made was alleged to have been unprofes...  0.507718
14  complainant accused officer of disch

In [14]:
model.save("models/noso")

In [15]:
# model.generate_topic_wordcloud(0)

In [16]:
model.topic_words

array([['an', 'complainant', 'officer', 'not', 'with', 'accused',
        'using', 'in', 'of', 'while', 'documented', 'phone', 'complaint',
        'who', 'action', 'to', 'altercation', 'for', 'off', 'no', 'did',
        'on', 'criminal', 'found', 'during', 'the', 'out', 'violation',
        'her', 'proper', 'incident', 'taken', 'supervisor', 'parish',
        'deputy', 'which', 'failed', 'was', 'assigned', 'has',
        'arrested', 'that', 'sustained', 'about', 'it', 'his', 'contact',
        'against', 'rude', 'by'],
       ['body worn', 'activate', 'worn', 'uof', 'camera', 'end', 'body',
        'uof incident', 'an inmate', 'inmates', 'battery', 'medical',
        'shift', 'completing', 'during', 'spray', 'firearm', 'oc',
        'tour', 'inmate', 'instructed', 'phone', 'assignment',
        'revealed', 'tone', 'failed', 'language', 'opso', 'speaking',
        'all', 'through', 'assigned', 'record', 'pursuit', 'appear',
        'bwc', 'son', 'pod', 'handling', 'cell', 'using', 'gir

In [17]:
# model.topic_words[0]

In [18]:
topic_words, word_scores, topic_nums = model.get_topics(0)
for words, scores, nums in zip(topic_words, word_scores, topic_nums):
  print("Topic Number: ",nums)
  print(f"Words: {words}")
  print("\n")

In [19]:
# topic_words, word_scores, topic_scores, topic_nums = model.search_topics(keywords=["inmate"], num_topics=0)
# for word, w_score, topic, t_score in list(zip(topic_words, word_scores, topic_scores, topic_nums)):
#     print(f"Word: \n{word}")
#     print("--------------------")
#     print(f"Word Score \n{w_score}")
#     print("--------------------")
#     print(f"Topic Score: \n{topic}")
#     print("--------------------")
#     print (f"Topic # \n{t_score}")
#     print("--------END---------")

In [20]:
# model = Top2Vec.load("models/noso")

# umap_args = {
#     "n_neighbors": 15,
#     "n_components": 2, # 5 -> 2 for plotting 
#     "metric": "cosine",
# }
# umap_model = umap.UMAP(**umap_args).fit(model.topic_vectors)
# umap.plot.points(umap_model, labels=model.doc_top_reduced)

In [21]:
############################################################ bert #################################################################

In [22]:
docs = df.allegation_desc

In [23]:
docs = json.loads(docs.to_json(orient='records'))

In [24]:
topic_model = BERTopic(embedding_model="all-MiniLM-L6-v2")

In [25]:
topic, probs = topic_model.fit_transform(docs)

In [26]:
topic_model.get_topic_info()

,Topic,Count,Name
0,0,684,0_the_to_complainant_officer
1,1,16,1_late_duty_for_reported


In [27]:
# topic_model.get_topic(3)

In [28]:
# topic_model.get_representative_docs(3)

In [29]:
# topic_model.visualize_topics()

In [30]:
# topic_model.visualize_barchart()

In [31]:
df_bert = pd.DataFrame({"topic": topic, "documents": docs})

In [32]:
df_bert

,topic,documents
0,0,the complainant says there is a 2005 marked no...
1,0,failed to take necessary actions when escortin...
2,0,cmt reynolds accused of not retuning back to d...
3,0,complainant stated that he observed a mounted ...
4,0,officer failed to receive permission from an o...
...,...,...
695,0,complainant alleged the officer refused to tak...
696,0,failed to activate body worn camera for uof in...
697,0,accused was allegedly rude to complainant.
698,0,the officers were accused of not preparing an ...


In [33]:
############################################################ gensim ##############################################################################

In [34]:
train_docs = df.allegation_desc

In [35]:
def lemmatization(descs, allowed_pos_tags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    final_text = []
    for desc in descs:
        doc = nlp(desc)
        new_text = " ".join([token.lemma_ for token in doc if token.pos_ in allowed_pos_tags])
        final_text.append(new_text)
    return (final_text)

In [36]:
lemmatized_texts = lemmatization(docs)

In [37]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

In [38]:
data_words = gen_words(lemmatized_texts)

In [39]:
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return list(bigram[doc] for doc in texts)

def make_trigrams(texts):
    return list(trigram[bigram[doc]] for doc in texts)

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_words)

# print(data_bigrams_trigrams[5])

In [40]:
id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

train_corpus = [id2word.doc2bow(text) for text in texts]
# print(train_corpus[0][0:20])

tdidf = TfidfModel(train_corpus, id2word=id2word)

low_value = 0.03
words = []
words_missing_in_tdif = []

for i in range(0, len(train_corpus)):
    bow = train_corpus[i]
    low_value_words = []
    tdif_ids = [id for id, value in tdidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tdidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tdif
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tdif = [id for id in bow_ids if id not in tdif_ids]

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tdif]
    train_corpus[i] = new_bow

In [41]:
# id2word = corpora.Dictionary(data_words)

# corpus = []
# for text in data_words:
#     new = id2word.doc2bow(text)
#     corpus.append(new)

In [42]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=train_corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                        #    per_word_topics=True,
                                           alpha="auto")
lda_model.save('models/lda_train.model')

In [43]:
# lda_model.print_topics(5, num_words=20)[:10]


In [44]:
def lda_vecs():
    lda_train_vecs = []
    for i in range(len(train_docs)):
        top_topics = lda_model.get_document_topics(train_corpus[i], minimum_probability=0.0)
        topic_vec = [top_topics[i][1] for i in range(20)]
        topic_vec.extend([len(train_docs.iloc[i])]) # length review
        lda_train_vecs.append(topic_vec)
    return lda_train_vecs

In [45]:
lda_vecs = lda_vecs()
lda_vecs = np.array(lda_vecs)
lda_vecs = pd.DataFrame(lda_vecs).to_csv("vecs/lda_vecs.csv")

In [46]:
lda_vecs

In [47]:
lda_vecs

In [48]:
# test_doc = train_corpus[-1]

# vector = lda_model[test_doc]

# def Sort(sub_li):
#     sub_li.sort(key = lambda x: x[1])
#     sub_li.reverse()
#     return (sub_li)

# new_vector = Sort(vector)
# print(new_vector)

In [49]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, train_corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.364037 -0.267557       1        1  21.393089
0     -0.446584 -0.046063       2        1  20.093397
2     -0.210392 -0.305166       3        1   8.468253
3     -0.043053 -0.363202       4        1   7.666051
10     0.101095  0.375594       5        1   4.834615
15    -0.184083  0.324122       6        1   4.374042
13    -0.265722  0.175394       7        1   3.896639
5     -0.200286 -0.053159       8        1   3.475457
1      0.109524 -0.270198       9        1   3.444723
4      0.276312 -0.250995      10        1   3.415328
8      0.365611 -0.076794      11        1   3.235810
6      0.339758  0.187558      12        1   3.233861
12    -0.184531  0.105450      13        1   2.937244
11     0.012451 -0.145060      14        1   2.525158
14     0.001157  0.280105      15        1   2.385714
16     0.151400  0.186272      16        1   1.461353
18     0.221029 -0.021820      17        1   1.375686
19     0.168526  0.083113      18        1   0.944296
9      0.088814  0.024210      19        1   0.448768
17     0.063009  0.058197      20        1   0.390517, topic_info=             Term        Freq       Total Category  logprob  loglift
19         accuse  367.000000  367.000000  Default  30.0000  30.0000
32          state  360.000000  360.000000  Default  29.0000  29.0000
14           fail  234.000000  234.000000  Default  28.0000  28.0000
30        officer  772.000000  772.000000  Default  27.0000  27.0000
1     complainant  639.000000  639.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
635        school    0.018115    0.941057  Topic20  -7.5191   1.5952
518   subordinate    0.018115    0.941062  Topic20  -7.5191   1.5952
776         allow    0.018116    0.941197  Topic20  -7.5191   1.5951
1025     handcuff    0.018116    0.941218  Topic20  -7.5191   1.5951
624        submit    0.018115    0.941133  Topic20  -7.5191   1.5951

[749 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
317       8  0.468295    accept
431       6  0.969083  accident
206       3  0.783805    accord
19        2  0.998284    accuse
319      18  0.862652       act
...     ...       ...       ...
405      12  0.559597    worker
70        2  0.788488     write
70        3  0.212285     write
143      15  0.837956     wrong
361      14  0.644194       wss

[464 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 1, 3, 4, 11, 16, 14, 6, 2, 5, 9, 7, 13, 12, 15, 17, 19, 20, 10, 18])